Question 1: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

    - what are the characteristics and behaviours that have the strongest correlation with subscribing

Question 2: We would like to know which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts.

    - within the people who subscribed, what are the portfolios of the players?

Question 3: We are interested in demand forecasting, namely, what time windows are most likely to have large number of simultaneous players. This is because we need to ensure that the number of licenses on hand is sufficiently large to accommodate all parallel players with high probability. 

    - graph histogram 

In [ ]:
library(tidyverse)

**1. Data description**
- Provide a full descriptive summary of the dataset, including 
(1) information such as the **number of observations**, **summary statistics** (report values to 2 decimal places), **number of variables**, **name and type of variables**, what the **variables mean**
(2) any **issues** you see in the data, any other potential issues related to things you cannot directly see, how the data were collected, etc. Make sure to use bullet point lists or tables to summarize the variables in an easy-to-understand format.

Note that the selected dataset(s) will probably contain more variables than you need. In fact, exploring how the different variables in the dataset affect your model may be a crucial part of the project. You need to summarize the full data regardless of which variables you may choose to use later on.

In [ ]:
player_data <- read_csv("data/players.csv")
session_data <- read_csv("data/sessions.csv")
joined_data <- player_data |> left_join(session_data) |> group_by(hashedEmail) |> summarize(count = n())
summary_statistics_age <- player_data |> summarize(mean = mean(Age, na.rm = TRUE), max = max(Age, na.rm = TRUE), min = min(Age, na.rm = TRUE)) 

summary_statistics_hours1 <- player_data |> summarize(mean = mean(played_hours, na.rm = TRUE)) 
summary_statistics_hours2 <- player_data |> summarize(max = max(played_hours, na.rm = TRUE), min = min(played_hours, na.rm = TRUE)) 
summary_statistics_hours <- bind_cols(summary_statistics_hours1, summary_statistics_hours2)

summary_statistics_log <- joined_data |> summarize(mean = mean(count), max = max(count), min = min(count))

measure <- c("age", "played hours", "log times")

summary_statistics <- bind_rows(summary_statistics_age, summary_statistics_hours, summary_statistics_log) |> add_column(measure) |> relocate(measure, .before = mean)

summary_statistics

- Number of observations: There are 196 observations (players), as seen by the number of rows in the players.csv file. 
- Summary statistics: the average age of players in the game is around 21 years old, they usually played around 6 hours, and logged onto the game around 8 times.
The oldest player in the game is 58 years old. The player who spent the most time on the game spent 223.1 hours, and the player who logged in the most times logged onto the game 310 times (1 log is 1 session).
The youngest player in the game is 9 years old. The player who spent the minimum time on the game spent 0 hours, and the player who logged in the least amount of times logged onto the game once.
- name and type of variables: in the players.csv, there are 7 variables (Age, hashedEmail, name, subscribe, experience, played_hours, and gender). Each variable is a characteristic of a player: their age, their email, name, level of experience, gender, their played hours, and whether or not they are subscribed to the newsletter. In the sessions.csv, there are 5 variables (hashedEmail, start_time, end_time, original_start_time, original_end_time). Each variable is a characteristic of a session: the player in the session, the start time and end time of the session, and the original start and end time of that session. 
- Some issues that arise are that there are NA values, for example, in Age, meaning the research group is not able to collect all the details of all players. Another thing is how "player" is defined, whether one is considered a player when making an account, or a player when they start to play. Many players have 0 hours of playtime, and whether or not they are taken into consideration is important because this changes the value of the average time spent on the game dramatically (if they are considered, the mean is around 6 hours, and if they aren't, the mean is around 10 hours). A final issue could be related to ethics, in terms of how the research group may use this collected data without breaching the privacy of their players. 

**2. Question**
- Clearly state **one broad question** that you will address, and the **specific question** that you have formulated. Your question should involve **one response variable of interest and one or more explanatory variables**, and should be stated as a question. One common question format is: “Can [explanatory variable(s)] predict [response variable] in [dataset]?”, but you are free to format your question as you choose so long as it is clear. **Describe** clearly how the data will help you address the question of interest. You may need to describe how you plan to wrangle your data to get it into a form where you can apply one of the predictive methods from this class.

- I will be choosing question 2: which "kinds" of players are most likely to contribute a large amount of data? 
- Therefore, my specific question will be: Can the player's age and hours spent on the game predict their level of expertise (experience)? 
- The data will help me address my question because the players' age, level of experience, and hours spent on the game are all recorded. Additionally, there are a large number of observations (196), which will allow us to split the data into multiple smaller datasets and hence train and test our predictive models (as compared to a dataset with only 10 observations, for example). I plan to change the column "experience" from "character" to "factor" to convert the values into labels that models, such as the KNN classification model, can use. 

**3. Exploratory data analysis and visualization**
- In this assignment, you will: Demonstrate that the dataset can be loaded into R.
- Do the minimum necessary wrangling to **turn your data into a tidy format**. Do not do any additional wrangling here; that will happen later during the group project phase.
- Compute the **mean value for each quantitative variable in the players.csv data set**. Report the mean values in a table format. Make a few exploratory visualizations of the data to help you understand it.
- Use our visualization best practices to make **high-quality plots** (make sure to include labels, titles, units of measurement, etc)
- **Explain any insights** you gain from these plots that are relevant to address your question
Note: do not perform any predictive analysis here. We are asking for an exploration of the relevant variables to demonstrate that you understand them well before performing any additional modelling, and to identify potential problems you anticipate encountering.

In [ ]:
summary_statistics_3 <- summary_statistics |> filter(measure %in% c("played hours" , "age"))
summary_statistics_3
plot_for_age <- player_data |> ggplot(aes(x = Age)) + geom_histogram(binwidth = 4, position = "stack") + scale_y_log10() + labs(x = "Age of the player", y = "Number of players (log(number))", title = "The distribution of players by age")
plot_for_age

plot_for_played_hours <- player_data |> ggplot(aes(x = played_hours)) + geom_histogram() + scale_y_log10() + scale_x_log10()  + labs(x = "Number of hours played (log(hour))", y = "Number of players (log(number))", title = "The distribution of players by their hours spent on the game")
plot_for_played_hours

plot_for_experience <- player_data |> group_by(experience) |> summarize(count = n()) |> mutate(percentage = count/196*100) |> 
ggplot(aes(x = experience, y = percentage)) + geom_bar(stat = "identity") + labs(x = "The gaming level of the player", y = "Number of players", title = "The distribution of players by their gaming level")
plot_for_experience

plot_agevshoursvsexperience <- player_data |> ggplot(aes(x= Age, y = played_hours, color = experience)) + geom_point() + scale_y_log10()
plot_agevshoursvsexperience

    To answer the broad question:
- With the first histogram, it is clear that people in their teenage years and 20s are the target demographic of the game because they contribute the most to the dataset.
- With the 2nd histogram, a large number of people who logged into the game have never played it. The majority of people play very little, and very few dedicated players spend a lot of time on the game. The number of hours is difficult to approximate because they are scaled by log10 for the sake of visual clarity.
- With the barplot, players' level is very polarised. While the majority (more than 30%) of players are amateurs, there is also a nearly as large number of players who are veterans (around 25%). The rest vary from beginner to regular.

    To study the specific question:
- With the scatterplot, it is clear that there is no linear relationship between either of the variables. Most players are young (teens) with all sorts of levels of experience and amount of hours played. Therefore, when constructing a predictive model, it is prudent to use KNN classification because the model doesn't impose any shape of relationship (linear, circular, ...) on the data. 

**4. method and plan**
- Propose one method to address your question of interest using the selected dataset and explain why it was chosen. Do not perform any modelling or present results at this stage. We are looking for high-level planning regarding model choice and justifying that choice.

- In your explanation, respond to the following questions:

    - Why is this method appropriate?
    - Which assumptions are required, if any, to apply the method selected?
    - What are the potential limitations or weaknesses of the method selected?
    - How are you going to compare and select the model?
    - How are you going to process the data to apply the model? For example: Are you splitting the data? How? How many splits? What proportions will you use for the splits? At what stage will you split? Will there be a validation set? Will you use cross validation?

To answer the question "Can the player's age and hours spent on the game predict their level of expertise (experience)?", I will use KNN Classification. The predictors are "Age" and "played_hours" and the label to be predicted is "experience" (which I will change to a factor when tidying data). 

- I will split data into 75-25 (training-testing) because that's the 
- I will split the data into 3-5 subsets (because the dataset is not particularly large, there are only around 200 observations) and practice cross-validation to see which K yields the most accurate predictions.
- I will use 
- Because our data didn't show any linear relationship between variables, this method is appropriate because it doesn't require the data to follow specific patterns.
- The potential limitation is our imbalanced classes: there are more amateurs and veterans than any other level of experience, which could make our model less accurate as many neighbours to a point are amateurs and veterans due to them simply being more represented.


**5. github repository**